In [3]:
import sys

import matplotlib.pyplot as plt
import numpy as np

sys.path.append('../../../')
from utils.plot import figsize, fig_save_and_show, config_matplotlib
from helper import plot_hull_position

config_matplotlib()
latex_img_path = '/home/joaoantoniocardoso/workspace_TCC/repo/thesis/assets/generated/'
latex_tex_path = '/home/joaoantoniocardoso/workspace_TCC/repo/thesis/tex/generated/'

# Hydrostatics studies: exploring the _wetted surface area_ and _trim_ as functions of _center of gravity_ and _displacement mass_.

## Load the geometry file
![](models/hull.jpeg "Hull.stl")

In [ ]:
### Load the geometry file
# !pip install https://github.com/LHEEA/meshmagick/archive/refs/tags/3.4.zip
import os
from meshmagick import mmio, mesh, hydrostatics

path = os.getcwd() + "/data/"
file = "hull_decimated.stl"

V, F = mmio.load_STL(path + file)
mymesh = mesh.Mesh(V, F)

# Check if the mesh is closed too:
print("Is it closed?", mymesh.is_mesh_closed())

# And inspect the mesh quality
mymesh.print_quality()

## Simulation with the hydrostatics solver
With the Mesh loaded we can create a simulation (using the hydrostatics solver) to get data to model the hull behaviour. We want to set the **displacement mass** (in TONS) and its **c.o.g.** (center of gravity) location ($x, y, z$ in meters, relative to the model's origin), and get the hull resulting **trim** and its **wetted surface area**.

In [ ]:
### Simulate:
# !pip install tqdm ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.spatial import ConvexHull
from meshmagick import densities
import multiprocessing as mp
from IPython.utils import io
from tqdm.autonotebook import tqdm


def _waterline_metrics(verts, atol=1e-2):
    wl_mask = np.isclose(verts[:, 2], 0.0, atol=atol)
    if np.any(wl_mask):
        wl_points = verts[wl_mask][:, :2]
        bwl = float(wl_points[:, 1].max() - wl_points[:, 1].min())
        lwl = float(wl_points[:, 0].max() - wl_points[:, 0].min())
        try:
            awp = float(ConvexHull(wl_points).volume)
        except Exception:
            awp = float(0.0)
    else:
        bwl = float(verts[:, 1].max() - verts[:, 1].min())
        lwl = float(verts[:, 0].max() - verts[:, 0].min())
        awp = float(0.0)
    return bwl, lwl, awp


def simulate(mymesh, cog_x, mass, verbose=True):
    kg_to_tons = 1000
    disp = mass / kg_to_tons
    cog = np.array([cog_x, 0, 0])
    water_density = densities.get_density("SALT_WATER")
    grav = 9.81
    reltol = 1e-4
    z_corr, rotmat_corr = hydrostatics.full_equilibrium(
        mymesh,
        cog,
        disp,
        water_density,
        grav,
        reltol=reltol,
        verbose=verbose,
    )
    hs_data = hydrostatics.compute_hydrostatics(
        mymesh,
        cog,
        water_density,
        grav,
        z_corr=z_corr,
        rotmat_corr=rotmat_corr,
        at_cog=True,
    )

    mymesh.rotate_matrix(rotmat_corr)
    mymesh.translate_z(z_corr)

    bwl, lwl, awp = _waterline_metrics(mymesh.vertices, atol=1e-2)

    r = R.from_matrix(hs_data["rotmat_eq"])
    angles_deg = r.as_euler("xyz", degrees=True)

    return {
        "disp_mass": hs_data["disp_mass"],
        "cog_x": hs_data["cog"][0],
        "cog_y": hs_data["cog"][1],
        "cog_z": hs_data["cog"][2],
        "wet_surface_area": hs_data["wet_surface_area"],
        "cob_x": hs_data["buoyancy_center"][0],
        "cob_y": hs_data["buoyancy_center"][1],
        "cob_z": hs_data["buoyancy_center"][2],
        "angles_deg_x": angles_deg[0],
        "angles_deg_y": angles_deg[1],
        "angles_deg_z": angles_deg[2],
        "LWL": lwl,
        "BWL": bwl,
        "AWP": awp,
    }


def parallel_simulate(args):
    VF, data = args
    mymesh = mesh.Mesh(*VF)

    with io.capture_output() as _:  # Supress every text output from simulate
        try:
            result = simulate(mymesh, cog_x=data[0], mass=data[1], verbose=False)
        except Exception:
            result = None
    return result


In [ ]:
### Run the simulations grid
VF = (V, F)

cog_x_grid = np.linspace(1.0, 4.0, 100)
disp_mass_grid = np.linspace(100, 400, 100)
grid = [(cx, m) for cx in cog_x_grid for m in disp_mass_grid]

with mp.Pool(mp.cpu_count()) as pool:
    results = list(
        tqdm(pool.imap(parallel_simulate, [(VF, g) for g in grid]), total=len(grid))
    )


In [ ]:
### Store the simulations's output to a file:
df = pd.DataFrame([r for r in results if r])
df.to_feather("data/meshmagick.feather")

## Operating point params

In [1]:
import os
import numpy as np
from meshmagick import mmio, mesh, hydrostatics
from meshmagick.mesh_clipper import MeshClipper
from meshmagick.mesh import Plane

# --- Load and clean mesh ---
path = os.getcwd() + '/data/'
file = "hull_decimated.stl"

V, F = mmio.load_STL(path + file)
mymesh = mesh.Mesh(V, F)

print(f"Loaded mesh: {mymesh.nb_faces} faces")
print(f"isclosed? {mymesh.is_mesh_closed()}")

# Aggressive cleanup
# mymesh.heal_mesh()
mean_area = np.mean(mymesh.faces_areas)
mymesh.remove_degenerated_faces(rtol=1e-2)  # Remove very small faces
mymesh.merge_duplicates(atol=1e-5)
print(f"After cleanup: {mymesh.nb_faces} faces")

# --- Remove deck ---
face_centers = mymesh.faces_centers
face_normals = mymesh.faces_normals
z_max = np.max(face_centers[:, 2])
deck_mask = (face_centers[:, 2] > (z_max - 1e-3)) & (face_normals[:, 2] > 0.9)
hull_mesh = mymesh.extract_faces(np.where(~deck_mask)[0])

# hull_mesh.heal_mesh()
print(f"Hull-only: {hull_mesh.nb_faces} faces, closed? {hull_mesh.is_mesh_closed()}")

# --- Align to waterplane ---
z_top = np.max(hull_mesh.vertices[:, 2])
hull_mesh.translate([0, 0, -z_top])
# hull_mesh.show()

# --- Clip just below deck edge ---
waterplane = Plane(normal=[0, 0, 1], scalar=1e-6)  # Slightly below z=0
clipper = MeshClipper(hull_mesh, plane=waterplane, assert_closed_boundaries=False, verbose=False)
wet_area = np.sum(clipper.clipped_mesh.faces_areas)
total_area = np.sum(hull_mesh.faces_areas)

print(f"\n✅ Total hull area: {total_area:.6f} m²")
print(f"Wetted area (at deck edge): {wet_area:.6f} m²")
print(f"error: {abs(total_area - wet_area):.2e} m²")

Loaded mesh: 3058 faces
isclosed? True
After cleanup: 2991 faces
FOUND OPEN BOUNDARY!!!
Hull-only: 2621 faces, closed? False

✅ Total hull area: 8.238401 m²
Wetted area (at deck edge): 8.238401 m²
error: 0.00e+00 m²


In [4]:
from utils.optimization import save_model_params_to_json

hull_params = dict(
    total_area=total_area,
    cog_x=1.9760,
    disp_mass=293.7,
)

save_model_params_to_json(path + 'hull_input_params.json', hull_params)

# Hull Total Surface Area

In [ ]:
import os
import numpy as np
from meshmagick import mmio, mesh, hydrostatics
from meshmagick.mesh_clipper import MeshClipper
from meshmagick.mesh import Plane

# --- Load and clean mesh ---
path = os.getcwd() + '/data/'
file = "hull.stl"

V, F = mmio.load_STL(path + file)
mymesh = mesh.Mesh(V, F)

print(f"Loaded mesh: {mymesh.nb_faces} faces")
print(f"isclosed? {mymesh.is_mesh_closed()}")

# Aggressive cleanup
# mymesh.heal_mesh()
mean_area = np.mean(mymesh.faces_areas)
mymesh.remove_degenerated_faces(rtol=1e-2)  # Remove very small faces
mymesh.merge_duplicates(atol=1e-5)
print(f"After cleanup: {mymesh.nb_faces} faces")

# --- Remove deck ---
face_centers = mymesh.faces_centers
face_normals = mymesh.faces_normals
z_max = np.max(face_centers[:, 2])
deck_mask = (face_centers[:, 2] > (z_max - 1e-3)) & (face_normals[:, 2] > 0.9)
hull_mesh = mymesh.extract_faces(np.where(~deck_mask)[0])

# hull_mesh.heal_mesh()
print(f"Hull-only: {hull_mesh.nb_faces} faces, closed? {hull_mesh.is_mesh_closed()}")

# --- Align to waterplane ---
z_top = np.max(hull_mesh.vertices[:, 2])
hull_mesh.translate([0, 0, -z_top])
# hull_mesh.show()

# --- Clip just below deck edge ---
waterplane = Plane(normal=[0, 0, 1], scalar=1e-6)  # Slightly below z=0
clipper = MeshClipper(hull_mesh, plane=waterplane, assert_closed_boundaries=False, verbose=False)
wet_area = np.sum(clipper.clipped_mesh.faces_areas)
total_area = np.sum(hull_mesh.faces_areas)

print(f"\n✅ Total hull area: {total_area:.6f} m²")
print(f"Wetted area (at deck edge): {wet_area:.6f} m²")
print(f"error: {abs(total_area - wet_area):.2e} m²")

## Load, Filter and Plot
To avoid some inconsistency on the dataset (for example, when the simulation doesn't converge), we can filter any abrupt angle in Y axis. We can consider only angles less than 10°.

In [ ]:
### Plot:
# %matplotlib notebook
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('default')

def load_dataset():
    df = pd.read_feather('data/meshmagick.feather')
    df = df[abs(df['angles_deg_y']) < 10].reset_index(drop=True)
    return df

def plot_dataset(df: pd.DataFrame):
    fig = plt.figure(figsize=plt.figaspect(1/2))

    X = df['cog_x']
    Y = df['disp_mass']

    Z = df['angles_deg_y']
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    ax.plot_trisurf(X, Y, Z, cmap='Greens', edgecolor=None, alpha=0.7)
    ax.scatter(X, Y, Z, c='black', s=1)
    ax.set_xlabel('CoG [m]')
    ax.set_ylabel('Mass [kg]')
    ax.set_zlabel('Trim [Deg]')
    ax.view_init(elev=30, azim=-120)

    Z = df['wet_surface_area']
    ax = fig.add_subplot(1, 2, 2, projection='3d')
    ax.plot_trisurf(X, Y, Z, cmap='Blues', edgecolor=None, alpha=0.7)
    ax.scatter(X, Y, Z, c='black', s=1)
    ax.set_xlabel('CoG [m]')
    ax.set_ylabel('Mass [kg]')
    ax.set_zlabel('Wet Area [m²]')
    ax.view_init(elev=30, azim=-120)

    plt.show()

df = load_dataset()
display(df)
plot_dataset(df)

df.describe().T

In [ ]:
%matplotlib inline
import seaborn as sns
sns.set_theme()

sns.pairplot(df[['cog_x', 'disp_mass', 'angles_deg_y', 'wet_surface_area']], hue='cog_x')
plt.show()

sns.pairplot(df[['cog_x', 'disp_mass', 'angles_deg_y', 'wet_surface_area']], hue='disp_mass')
plt.show()

### Observations
- The greater the **displacement mass**, greater the **wetted surface area**;
- The **displacement mass** doesn't correlate well with the **y angle (trim)**;
- The greater the **cog x**, greater the **y angle (trim)**;
- The lowest **wetted surface area** occur with the **displacement mass**, but also with higher **cog x**;

## Visually understanding the change in CoG:

#### Case 1:
- mass = 265kg
- cog_x = 1m

![](data/hull_265kg_1m.jpeg)

#### Case 2:
- mass = 265kg
- cog_x = 4m

![](data/hull_265kg_4m.jpeg)

In [ ]:
def plot_hull_position(result, mesh):
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.tri import Triangulation
    from utils.plot import figsize

    V_eq = mesh.vertices
    triangles = np.asarray(getattr(mesh, "faces", None))[:, :3].astype(int)

    tri = Triangulation(V_eq[:, 0], V_eq[:, 2], triangles)

    fig, ax = plt.subplots(
        figsize=figsize(subplots=(1, 2)),
        constrained_layout=True
    )

    ax.triplot(tri, color="gray", linewidth=0.2, alpha=1, label="Hull")
    ax.axhline(0.0, color="blue", linewidth=2, alpha=0.5, label="Waterplane")

    cog_x = result.get("cog_x", 0.0)
    cog_z = result.get("cog_z", 0.0)
    ax.scatter([cog_x], [cog_z], color="red", s=20, marker="^")

    ax.set_xlabel("x [m]")
    ax.set_ylabel("z [m]")

    # critical: prevent geometric distortion
    ax.set_aspect("equal", adjustable="box")

    return fig


In [ ]:
import os
from meshmagick import mmio, mesh

path = os.getcwd() + "/data/"
file = "hull_decimated.stl"


for mass in [100, 265, 350]:
    for cog_x in [1, 2, 3, 4]:
        V, F = mmio.load_STL(path + file)
        mymesh = mesh.Mesh(V, F)

        try:
            result = simulate(mymesh, mass=mass, cog_x=cog_x)
        except:
            continue
        display(result)

        fig_save_and_show(
            filename=None, #'{latex_img_path}/battery_optimization_dataset.pdf',
            show_title=f'Hull at equilibrium for {mass} [kg] at {cog_x} [m]',
            save_title=f'Casco em equilíbrio para {mass} [kg] em {cog_x} [m]',
            ncol=4,
            fig=plot_hull_position(result, mymesh),
        )